In [21]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import sklearn
from sklearn.cluster import KMeans

In [9]:
##Client Authorization

cid = "26e46f3a7818494286f6349bd492484d"
secret = "f96866fd97af42b38349b865821e631f"
ruri = "http://localhost:64664/callback"

scope = "user-library-read"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret = secret)
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id = cid, client_secret = secret, redirect_uri = ruri))

In [10]:
##function to add tracks to a dataframe
def track_adder(dataframe, track_dict): ##where dataframe is the master dataframe and track_dict a dictionary with structure {playlist title: [uri, mood]}
    for playlist in track_dict:
        plist = sp.playlist_tracks(track_dict[playlist][0]) ##gets the json entry for the playlist from the api
        i = 0
        while i < len(plist["items"]): ##crawl through the tracks in items
            dataframe = dataframe.append({"track": plist["items"][i]["track"]["name"], "uri": plist["items"][i]["track"]["uri"], "mood": track_dict[playlist][1]}, ignore_index = True) ##adds each track info to the dataframe
            i = i+1
    return dataframe

In [11]:
##function to add track features to data frame
def feature_adder(dataframe): ##just needs a data frame 
    for index, row in dataframe.iterrows():
        features = sp.audio_features(row[1]) ##pulls the audio features and fills them in accordingly
        row[3] = features[0]["danceability"]
        row[4] = features[0]["energy"]
        row[5] = features[0]["key"]
        row[6] = features[0]["loudness"]
        row[7] = features[0]["mode"]
        row[8] = features[0]["speechiness"]
        row[9] = features[0]["acousticness"]
        row[10] = features[0]["instrumentalness"]
        row[11] = features[0]["liveness"]
        row[12] = features[0]["valence"]
        row[13] = features[0]["tempo"]
    return dataframe

In [12]:
##function to create our database
def create_database(trackDict): ##track_dict a dictionary with structure {playlist title: [uri, mood]}
    tracks = pd.DataFrame(columns = ["track", "uri", "mood", "danceability", "energy", "key","loudness", "mode", "speechiness", "acousticness", "instrumentalness", "liveness", "valence", "tempo"], index = None)
    tracks = track_adder(tracks, trackDict)
    tracks = feature_adder(tracks)
    return tracks

In [13]:
playlistData = {"happy hits": ["spotify:playlist:37i9dQZF1DXdPec7aLTmlC", "happy"], "good vibes": ["spotify:playlist:37i9dQZF1DWYBO1MoTDhZI", "happy"], "alone again": ["spotify:playlist:37i9dQZF1DWX83CujKHHOn", "sad"], "sad songs": ["https://open.spotify.com/playlist/37i9dQZF1DX7qK8ma5wgG1?si=c5E8sN-GQhCn_qevPW-9OQ","sad"]}

In [14]:
tracksData = create_database(playlistData)

In [19]:
def returner(df):
    return df

In [20]:
returner(tracksData)

,track,uri,mood,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,1950,spotify:track:0CZ8lquoTX2Dkg7Ak2inwA,happy,0.6,0.535,4,-7.043,1,0.107,0.648,4.7e-06,0.162,0.269,71.912
1,Love You For A Long Time,spotify:track:6QTPacyXkZWG9FMwq6L1hJ,happy,0.707,0.67,9,-6.137,1,0.0323,0.164,0.00283,0.109,0.822,112.993
2,Way Less Sad,spotify:track:3cwaaelWghY5W6Lsj1K8ss,happy,0.701,0.547,2,-8.238,1,0.0483,0.00843,0,0.0987,0.4,140.015
3,I Forgot That You Existed,spotify:track:43rA71bccXFGD4C8GOpIlN,happy,0.664,0.316,5,-10.345,1,0.519,0.298,2.03e-06,0.0812,0.541,92.875
4,Wonder,spotify:track:5KCbr5ndeby4y4ggthdiAb,happy,0.333,0.637,1,-4.904,0,0.0581,0.131,1.8e-05,0.149,0.132,139.898
5,Put Your Records On,spotify:track:1fah1uAs7HeTYDlNftKr3K,happy,0.399,0.492,6,-10.777,0,0.0538,0.058,0.00101,0.11,0.159,91.066
6,90s Kids,spotify:track:34evGV6SaoRLHTcXXUVOpb,happy,0.714,0.657,0,-6.436,1,0.118,0.0802,0,0.35,0.725,151.898
7,Mariposa,spotify:track:4ja2gzrNh9VNigzoXfmbwD,happy,0.676,0.525,9,-5.88,1,0.028,0.617,0,0.356,0.421,112.011
8,You (with Marshmello & Vance Joy),spotify:track:1GkHyypTFkUf0QQKwYoXH4,happy,0.668,0.563,11,-6.433,1,0.0522,0.0493,0,0.119,0.305,103.116
9,I Love Me,spotify:track:2IHLaN6wwYKQ3S24NsQqpA,happy,0.664,0.561,6,-7.031,1,0.0495,0.00261,0,0.0764,0.713,150.093


In [22]:
kmeansModel = sklearn.cluster.KMeans(n_clusters=2)
kmeans = kmeansModel.fit_predict(tracksData[["danceability","energy","key","loudness","mode","speechiness","acousticness","instrumentalness","valence","tempo"]])

In [23]:
kmeans = pd.DataFrame(kmeans, columns = ["kmeans"])
spotDataK = pd.concat([tracksData,kmeans], axis = 1)

In [26]:
group1 = spotDataK[spotDataK["kmeans"]==0]["mood"]
group2 = spotDataK[spotDataK["kmeans"]==1]["mood"]

In [28]:
happycount=0
sadcount=0
for row in group1:
    if row == "happy":
        happycount+=1
    elif row=="sad":
        sadcount+=1

print("group 1")
print("happy: "+str(happycount)+"\t"+str(happycount/group1.shape[0]))
print("sad: "+str(sadcount)+"\t\t"+str(sadcount/group1.shape[0]))


happycount=0
sadcount=0
for row in group2:
    if row == "happy":
        happycount+=1
    elif row=="sad":
        sadcount+=1

print("\ngroup 2")
print("happy: "+str(happycount)+"\t"+str(happycount/group2.shape[0]))
print("sad: "+str(sadcount)+"\t\t"+str(sadcount/group2.shape[0]))

group 1
happy: 150	0.7109004739336493
sad: 61		0.2890995260663507

group 2
happy: 50	0.5050505050505051
sad: 49		0.494949494949495
